#### 필요한 모듈 불러오기 ###

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

import os 

In [2]:
Data = pd.read_excel("01_data.xlsx")

# 데이터 분석을 용이하기 위해, 시간 정보를 datetime 형식으로 변경
Data['Time'] = pd.to_datetime(Data['Time'])
Data = Data.set_index('Time')

column_typ = ['X'+'{}'.format(i) for i in range(1,28)]

Train, Test = Data.iloc[:-7014,:], Data.iloc[-7014:,:] 
X_train, Y_train_01, Y_train_02 = Train.iloc[:, :-2], Train.iloc[:, -2], Train.iloc[:, -1]
X_test, y_01, y_02 = Test.iloc[:, :-2], Test.iloc[:, -2], Test.iloc[:, -1]

# 누락된 값 이전과 이후로 구분
part_01_x, part_01_y1, part_01_y2 = X_train.loc['2016-01-01':'2017-08-21'], Y_train_01.loc['2016-01-01':'2017-08-21'],  Y_train_02.loc['2016-01-01':'2017-08-21']
part_02_x, part_02_y1, part_02_y2 = X_train.loc['2017-10-05':],  Y_train_01.loc['2017-10-05':],  Y_train_02.loc['2017-10-05':]
X_train, y_train_01, y_train_02 = part_02_x, part_02_y1, part_02_y2 

In [3]:
from sklearn.linear_model import LinearRegression, Ridge, Lasso
from sklearn.metrics import r2_score,mean_absolute_error, mean_squared_error
from sklearn.model_selection import train_test_split
from sklearn.model_selection import cross_val_score
from sklearn.linear_model import Lasso

part_01 = pd.concat([X_train, y_train_01], axis = 1)
part_02 = pd.concat([X_train, y_train_02], axis = 1)

# Ridge
from bayes_opt import BayesianOptimization
from sklearn.model_selection import TimeSeriesSplit
from sklearn.model_selection import train_test_split, GridSearchCV, KFold
from sklearn.linear_model import Ridge
import warnings
warnings.filterwarnings('ignore')

from xgboost import XGBRegressor
from sklearn.ensemble import RandomForestRegressor

#### BayesianOptimization 사용

In [ ]:
from sklearn.linear_model import LinearRegression, Ridge, Lasso
from sklearn.metrics import r2_score,mean_absolute_error, mean_squared_error
from sklearn.model_selection import train_test_split
from sklearn.model_selection import cross_val_score
from sklearn.linear_model import Lasso

part_01 = pd.concat([X_train, y_train_01], axis = 1)
part_02 = pd.concat([X_train, y_train_02], axis = 1)

# Ridge
from bayes_opt import BayesianOptimization
from sklearn.model_selection import TimeSeriesSplit
from sklearn.model_selection import train_test_split, GridSearchCV, KFold
from sklearn.linear_model import Ridge
import warnings
warnings.filterwarnings('ignore')

from xgboost import XGBRegressor
from sklearn.ensemble import RandomForestRegressor

from sklearn.metrics import mean_absolute_error
from sklearn.metrics import r2_score

In [ ]:
x_tr, x_te, y_tr, y_te = train_test_split(X_train, y_train_01, test_size=0.2, random_state = 0, shuffle = True)
kfold = KFold(n_splits= 5, shuffle=True)


def xgb(max_depth,learning_rate, n_estimators, gamma ,min_child_weight, subsample, colsample_bytree, silent=True, nthread=-1):

    model = XGBRegressor(max_depth=int(max_depth),
                         learning_rate=learning_rate,
                         n_estimators=int(n_estimators),
                         gamma=gamma,
                         min_child_weight=int(min_child_weight),
                         subsample=subsample,
                         colsample_bytree=colsample_bytree, 
                         nthread=nthread,
                        n_jobs = -1)

    mse = []
    for train_index, test_index in kfold.split(x_tr):
        X_train, X_test = x_tr.iloc[train_index], x_tr.iloc[test_index]
        y_train, y_test = y_tr.iloc[train_index], y_tr.iloc[test_index]

        model.fit(X=X_train, y=y_train)

        predictions = model.predict(X=X_test)
        mse.append((mean_squared_error(y_test, predictions)))

    return -np.mean(mse)

pbounds = {'max_depth': (3, 15),
            'learning_rate': (0.01, 0.2),
            'n_estimators': (100, 1000),
            'gamma': (1, 10),
            'min_child_weight': (1, 5),
            'subsample': (0.5, 1),
            'colsample_bytree' :(0.2, 1)}

optimizer =  BayesianOptimization(f=xgb, pbounds=pbounds, random_state=0, allow_duplicate_points=True)
optimizer.maximize(init_points=10, n_iter=20, acq = 'ei', xi = 0.01)

# 최적화된 하이퍼파라미터로 모델 학습
best_params_ = optimizer.max['params']

print("Best parameter: {}".format(best_params_))

model = XGBRegressor(max_depth=int(best_params_['max_depth']) ,
                         learning_rate=np.round(best_params_['learning_rate'], 2),
                         n_estimators=int(best_params_['n_estimators']),
                         gamma= np.round(best_params_['gamma'],2),
                         min_child_weight=int(best_params_['min_child_weight']), 
                         subsample = np.round(best_params_['subsample'], 1),
                         colsample_bytree = np.round(best_params_['colsample_bytree'], 2),
                         n_jobs=-1, random_state=0)

model.fit(X=x_tr, y=y_tr)
y_pred = model.predict(x_te)

mse = mean_squared_error(y_te, y_pred)
mae = mean_absolute_error(y_te, y_pred)
r2 = r2_score(y_te, y_pred)
print("MSE: {} / MAE: {} / R2: {}".format(mse, mae, r2))

paper = pd.DataFrame(y_te, index=x_te.index)
pred = pd.DataFrame(y_pred, index=x_te.index)
pred = pred.rename(columns = {0 : pred_name})
value = pd.merge(paper, pred, left_index = True, right_index=True)

#전반부 에 대한 예측값과 전반부 실제값의 차이 시각화
pred_name = "Y_pred"
paper = pd.DataFrame(y_te, index=x_te.index)
pred = pd.DataFrame(y_pred, index=x_te.index)
pred = pred.rename(columns = {0 : pred_name})
value = pd.merge(paper, pred, left_index = True, right_index=True)

#전반부 에 대한 예측값과 전반부 실제값의 차이 시각화
fig, ax = plt.subplots(1,2,figsize = (20,5))
ax[0].plot(y_te.values, label='true')
ax[0].plot(y_pred, label='predicted')

ax[1].scatter(value.index, value["Y1"], label='y(validation)')
ax[1].scatter(value.index, value["Y_pred"], label='predicted y')
plt.show()